In [12]:
import timeit
import importlib
import relevancer as rlv
import pandas as pd
from sklearn.naive_bayes import MultinomialNB 
from bson.objectid import ObjectId
from collections import Counter

In [13]:
pd.set_option("display.max_colwidth",200)

In [67]:
importlib.reload(rlv)

<module 'relevancer' from '/Users/alihurriyetoglu/Dropbox/Projects/Relevancer/relevancer.py'>

In [15]:
my_token_pattern=r"[#@]?\w+\b|[\U00010000-\U0010ffff]"
collection = 'all_data'
rlvdb, rlvcl = rlv.connect_mongodb(configfile='myalldata.ini',coll_name=collection)
# set active columns
active_col = "active_text"
rlv.set_active_column(active_col)

In [16]:
#importlib.reload(rlv)
begin = ObjectId('55950fb4d04475ee9867f3a4')
end = ObjectId('55950fc9d04475ee986841c3')

# after the first iteration, the annotated clusters should be excluded from the clustering.
# read this file from the tagged collection.
annotated_tw_ids = ['563657483395530753', '563662532326330370', '563654330041909248', '563654944927281152', '563657924233289728', '563661021559390208', '563651950386757632', '563657164317667328', '563660271810383872', '563662538949160960'] #You should get the actual annotated tweet ids from the annotated tweets collection.
#annotated_tw_ids = []
# mongo_query=({'_id': {'$gte': begin, '$lte': end},'lang':'en'})
tweetlist = rlv.read_json_tweet_fields_database(rlvcl, mongo_query=({'lang':'en'}), tweet_count=50000, annotated_ids=annotated_tw_ids)#=tweetsDF)

rlv.logging.info("Number of tweets:" + str(len(tweetlist)))
print("Number of tweets:",len(tweetlist))

Number of tweets: 48524


In [297]:
importlib.reload(rlv)
begin = ObjectId('55950fb4d04475ee9867f3a4')
end = ObjectId('55950fc9d04475ee986841c3')

# after the first iteration, the annotated clusters should be excluded from the clustering.
# read this file from the tagged collection.
annotated_tw_ids = ['563657483395530753', '563662532326330370', '563654330041909248', '563654944927281152', '563657924233289728', '563661021559390208', '563651950386757632', '563657164317667328', '563660271810383872', '563662538949160960'] #You should get the actual annotated tweet ids from the annotated tweets collection.
fields_to_read = {'tweet_topic':1,'text': 1, 'id_str': 1, '_id': 0, 'user': 1}
#annotated_tw_ids = []
# mongo_query=({'_id': {'$gte': begin, '$lte': end},'lang':'en'})
rlv.read_json_tweet_fields_database(rlvcl, mongo_query=({'lang':'en'}), read_fields=fields_to_read, tweet_count=50000, annotated_ids=annotated_tw_ids)#=tweetsDF)

KeyboardInterrupt: 

In [17]:
tweetsDF = rlv.create_dataframe(tweetlist)

48524


In [18]:
# Be careful do not overwrite tweetsDFBackUP with a modified tweetsDF
#tweetsDFBackUP = tweetsDF.copy()
#len(tweetsDFBackUP)

In [251]:
# Get a clean copy 
importlib.reload(rlv)
tweetsDF = tweetsDFBackUP.copy()
len(tweetsDF)

48524

In [252]:
tweetsDF[active_col] = tweetsDF["text"]
tweetsDF = rlv.tok_results(tweetsDF, elimrt = True)


Available attributes of the tweets: Index(['id_str', 'is_retweet', 'screen_name', 'text', 'active_text',
       'texttok'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 48524 entries, 0 to 48523
Data columns (total 6 columns):
id_str         48524 non-null object
is_retweet     48524 non-null bool
screen_name    48524 non-null object
text           48524 non-null object
active_text    48524 non-null object
texttok        48524 non-null object
dtypes: bool(1), object(5)
memory usage: 2.3+ MB

tweet set summary: None
0    1st Test, DAY 2 - TEA:\n#IND 304/6 (91 ov, lead by 121, 1st inns)\nH Singh 0*,\nW Saha 25*,\n#SL 183.\n#CricbuzzCup\n#SLvIND
1       Younis Khan's 100th Test | Pakistan's go-to man gets to a landmark | Photo Gallery | ESPN Cricinfo http://t.co/ktnthao4hg
2                                            Test Engineer &amp;#x2013; DV Cleared #it #job, Manchester... http://t.co/wMQdnMfifS
3                                                            

/Users/alihurriyetoglu/Dropbox/Projects/Relevancer/relevancer.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  print(tweetsDF[tok_result_col][:5])


In [253]:
len(tweetsDF)

32977

In [254]:
tweetsDF = rlv.normalize_text(tweetsDF)
len(tweetsDF), tweetsDF.columns
#tweetsDF[[active_col]] #rlv.logging.info("This text overwritten by normalization" + str(tstDF[active_col]))

(32977,
 Index(['id_str', 'is_retweet', 'screen_name', 'text', 'active_text', 'texttok',
        'is_notrt'],
       dtype='object'))

In [301]:
importlib.reload(rlv)
rlv.set_active_column(active_col)

In [280]:
run_tweet_count = 5000
tweetsDF2 = rlv.get_and_eliminate_near_duplicate_tweets(tweetsDF[:run_tweet_count], similarity_threshold=0.20, debug=True, test_threshold=run_tweet_count)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2310 entries, 89 to 337
Data columns (total 7 columns):
id_str         2310 non-null object
is_retweet     2310 non-null bool
screen_name    2310 non-null object
text           2310 non-null object
active_text    2310 non-null object
texttok        2310 non-null object
is_notrt       2310 non-null bool
dtypes: bool(2), object(5)
memory usage: 112.8+ KB


In [281]:
tweetsDF3 = rlv.get_and_eliminate_near_duplicate_tweets(tweetsDF2, similarity_threshold=0.20, debug=True, test_threshold=run_tweet_count)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2233 entries, 265 to 2095
Data columns (total 7 columns):
id_str         2233 non-null object
is_retweet     2233 non-null bool
screen_name    2233 non-null object
text           2233 non-null object
active_text    2233 non-null object
texttok        2233 non-null object
is_notrt       2233 non-null bool
dtypes: bool(2), object(5)
memory usage: 109.0+ KB


In [282]:
tweetsDFx = rlv.get_and_eliminate_near_duplicate_tweets(tweetsDF, similarity_threshold=0.20, debug=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14878 entries, 140 to 36354
Data columns (total 7 columns):
id_str         14878 non-null object
is_retweet     14878 non-null bool
screen_name    14878 non-null object
text           14878 non-null object
active_text    14878 non-null object
texttok        14878 non-null object
is_notrt       14878 non-null bool
dtypes: bool(2), object(5)
memory usage: 726.5+ KB


In [287]:
tweetsDFx = rlv.get_and_eliminate_near_duplicate_tweets(tweetsDFx, similarity_threshold=0.20, debug=False)

There is not any group of near-duplicate tweets.


In [288]:
print("length at start:",len(tweetsDFx))
tweetsDFxx = rlv.get_and_eliminate_near_duplicate_tweets(tweetsDFx, similarity_threshold=0.20, debug=False)

length at start: 14279
There is not any group of near-duplicate tweets.


In [291]:
tweetsDFxx.columns

Index(['id_str', 'is_retweet', 'screen_name', 'text', 'active_text', 'texttok',
       'is_notrt'],
      dtype='object')

In [376]:
importlib.reload(rlv)
rlv.set_active_column(active_col)

In [408]:
cluster_list = rlv.create_clusters(tweetsDFxx, my_token_pattern, nameprefix='1-', min_clusters=1)

In [409]:
print("Number of clusters:",len(cluster_list))
print("available cluster information:", cluster_list[0].keys())

Number of clusters: 6
available cluster information: dict_keys(['cno', 'user_entropy', 'cnoprefix', 'ctweettuplelist', 'rif', 'cstr', 'twids'])


In [410]:
i = 0

In [419]:
print("No:",cluster_list[i]['cno'])
print("CStr:",cluster_list[i]['cstr'])

No: 64
CStr: cluster number and size are: 64    67



In [420]:
print("Cluster Tuple List:")
print(*[(c[0],c[2]) for c in cluster_list[i]['ctweettuplelist']], sep='\n')
# Add any field you want to observe.
i+=1

Cluster Tuple List:
(0.65718208865539485, 'hearthstone – test your knowledge with hearthstone quiz!\xa0#3 urlurlurl urlurlurl')
(0.686061211240381, 'test your knowledge &amp; answer the query ? urlurlurl')
(0.69910990092702874, 'test your knowledge this #fridayafternoon with our #quiz urlurlurl #fridaychallenge')
(0.70378298885456481, 'rtuusc: test your #climatechange knowledge: urlurlurl commit to #climate justice by joining commit2… urlurlurl')
(0.71391582833475586, 'internal audit - quiz to test your knowledge ! - urlurlurl urlurlurl')
(0.75375269906638098, 'test your knowledge of money with this "roth ira quiz" via @kiplinger urlurlurl #millennials #money')
(0.75777105167092662, "#fanoty - test your knowledge on history of premier league 'wonderkids' urlurlurl urlurlurl")
(0.76067765725186931, "quiz: test your news knowledge: test your knowledge of this week's news and current events. urlurlurl")
(0.7740038578358005, 'test your @goosebumpsmovie knowledge with this quiz and earn ent

In [414]:
print("Reverse Frequency of the terms:")
print(i)
print(*Counter(cluster_list[i-1]['rif']).most_common(), sep="\n")

Reverse Frequency of the terms:
1
('24', ['you', 'urlurlurl'])
('40', ['to'])
('2', ['those', 'took', 'tried', 'doing that', 'in a', 'to get', 'for a', 'who didn', 'need to', 'asking', 'i took', 'true test', '1', 'stop mark', 'test done', 'not doing', 've seen', 'chaffinch', 'in my', 'm still', 'listen', 'destiny', 'week', 'like me', 'time', 'appalling weather', 'i got', 'opened', 'during the', 'lie', 'weather didn', 't think', 'birthday', 'that you', 'damn', 'd', 'as i', 'drive', '@youtube', 'again', 'hope', 'it feels', 'mark', 'you wanted', 'name', 'i have', 'for your', 'test in', 'i feel', 'different', 'to my', 'appalling', 'passed the', 'body', 'were', 'it didn', 'sitting', 'get one', 'for the', 'doing', 'maybe', 'already', 'the end', 't stop', 'play', 'of your', 'weather', 'could have', 'for my', 'head', 'i ve', 'tell', 'took my', 'software', 've', 'goddamn', 'to go', 'driving test', 'thank u', 'then', 'want', 't need', 'to study', 'hear', 'mark passing', 'my driving', 'stop', 's 

In [31]:
collection_name = collection + '_clusters'
rlvdb[collection_name].insert(cluster_list) #Each iteration results with a candidate cluster list. Each iteration will have its own list. Therefore they are not mixed.
print("Clusters were written to the collection:", collection_name)

Clusters were written to the collection: coll123_clusters


In [26]:
tweets_as_text_label_df = pd.DataFrame({'label' : ['relif', 'social'] , 'text' : ["RT @OliverMathenge: Meanwhile, Kenya has donated Sh91 million to Malawi flood victims, according to the Ministry of Foreign Affairs." , "Yow ehyowgiddii! Hahaha thanks sa flood! #instalike http://t.co/mLaTESfunR"]})
print("tweets_as_text_label_df:", tweets_as_text_label_df)

tweets_as_text_label_df:     label                                               text
0   relif  RT @OliverMathenge: Meanwhile, Kenya has donat...
1  social  Yow ehyowgiddii! Hahaha thanks sa flood! #inst...


In [27]:
# get vectorizer and classifier
vect_and_classifier = rlv.get_vectorizer_and_mnb_classifier(tweets_as_text_label_df, my_token_pattern, pickle_file="vectorizer_and_classifier_dict")
vectorizer, mnb_classifier = vect_and_classifier["vectorizer"], vect_and_classifier["classifier"]

In get_mnb_classifier:
Pickle file was written to vectorizer_and_classifier_dict.pickle


In [ ]:
# get label for a new tweet:
ntw = vectorizer.transform(["Why do you guys keep flooding TL with smear campaign for a candidate you dont like.You think you can actually influnece people's decision?"])
predictions = mnb_classifier.predict(ntw)
print("Predictions:", predictions)

In [ ]:
rlv.logging.info('\nscript finished')

In [23]:
tw_id_list = rlv.get_ids_from_tw_collection(rlvcl)
print ("Length of the tweet ids and the first then ids",len(tw_id_list),tw_id_list[:10])

In get_tw_ids
Length of the tweet ids and the first then ids 48524 ['631754887861112832', '631754877526372352', '631754882907811841', '631754880378609664', '631754869959979008', '631754870232485888', '631754886422638592', '631754869347475458', '631754865627148288', '631754864675000320']
